In [ ]:
import numpy as np
import cv2
import time

In [ ]:
# to capture the feed from the webcam. The parameter for video capture is 0 because it corresponds to the primary inbuilt webcam on the machine.
cap = cv2.VideoCapture(0)
time.sleep(2)

In [ ]:
# loop to read each frame through the camera and capture the background

background = 0

for i in range(30):
    ret,background = cap.read()

In [ ]:
# capturing the image of the person with the cloak through each frame while the camera is open
while(cap.isOpened()):
     ret,img = cap.read()
     
     if not ret:
            break

# converting RGB to HSV, takes two parameters: which image is being operated on and the flag that specifies from which color space to which color space the image is being converted.
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

In [ ]:
# to detect the red color cloth in the frame, define the upper and lower bound to create a range. So the lower red is lightest values of red and upper values is the darkest values of red that are possible. Beyond this range the color is different.
# the mask is used to differentiate between the rest of the image and the cloak.
lower_red = np.array([0, 120, 70])
upper_red = np.array([10, 255, 255])
mask1 = cv2.inRange(hsv,lower_red, upper_red)

# now to differentiate between the cloak and the background. Red appears in these range of values as well so they need to be considered.
lower_red = np.array([170, 120, 70])
upper_red = np.array([180, 255, 255])
mask2 = cv2.inRange(hsv,lower_red, upper_red)

#now wherever the color red is found, that portion will be saved in the mask variables. Storing them both in one variable by the logical OR operator to make sure all red areas are accounted for.
mask1 = mask1 + mask2



In [ ]:
#segmenting using morphology and dilation. Reduces the noise present in feed and smoothens out the feed for a certain number of iterations
mask1 = cv2.morphologyEx(mask1, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations = 2)
mask1 = cv2.morphologyEx(mask1, cv2.MORPH_DILATE, np.ones((3,3), np.uint8), iterations = 1)

In [ ]:
# negating the mask2
mask2 = cv2.bitwise_not(mask1)

In [ ]:
#final result. res1 has the background part of the frame and res2 has the data for the cloak part of the frame

res1 = cv2.bitwise_and(background, background, mask=mask1)
res2 = cv2.bitwise_and(img, img, mask=mask2)


In [ ]:
# linearly adding the two results together to get the final output where we do alpha(1) times of res1, beta(1) times of res2, plus gamma(0)
final_output = cv2.addWeighted(res1,1,res2,1,0)

In [ ]:
#output, takes two parameters, the window title and the output variable.
cv2.imshow('invisibility cloak',final_output)

#close it when esc key is pressed. 27 is the ascii value of the escape key
k = cv2.waitKey(10)
if k==27:
    break

In [ ]:
cap.release()
cv2.destroyAllWindows()